In [1]:
import cv2
import tensorflow as tf
from keras.preprocessing import image
import numpy as np
import keras
from keras import Model
import time
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import Conv3D

fileName = "Trump.jpg"



# Global MobileNets model
mobile = keras.applications.mobilenet.MobileNet(weights="imagenet")
model = Model(mobile.input, mobile.layers[5].output)


Using TensorFlow backend.


In [ ]:
def write_to_file_input(fileName, X):
    fhandler = open(fileName, "w")
    for i in range(3):
        for j in range(224):
            for k in range(224):
                fhandler.write(str(X[j][k][i]) + "\n")
    fhandler.close()

In [ ]:
def create_Input_Files():
    
    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalizing the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Write Normalized Input to file
    write_to_file_input("data/FirstLayer/inputNorm.txt", X)
    
    
    # Write combination input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 10
    X[:, :, 2] = 100
    write_to_file_input("data/FirstLayer/inputComb.txt", X)
    
    # Write all 1's input to file
    X[:, :, 0] = 1
    X[:, :, 1] = 1
    X[:, :, 2] = 1
    write_to_file_input("data/FirstLayer/inputSet1.txt", X)

    print("Creating Input files Complete")


In [ ]:
create_Input_Files()

In [ ]:
def write_to_file_weights_layer_one(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(32):
        for j in range(3):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()
    

In [ ]:
W = model.layers[2].get_weights()[0]
write_to_file_weights_layer_one("data/FirstLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_one("data/FirstLayer/weightsSet1.txt", W)

In [ ]:
def write_to_file_weights_layer_second(fileName, W):
    fTemp = open(fileName, "w")
    for i in range(1):
        for j in range(32):
            for k in range(3):
                for l in range(3):
                    fTemp.write(str(W[k][l][j][i]) + "\n")
    fTemp.close()


In [ ]:
W = model.layers[5].get_weights()[0]
write_to_file_weights_layer_second("data/SecondLayer/weightsNorm.txt", W)

W[:,:,:,:] = 1
write_to_file_weights_layer_second("data/SecondLayer/weightsSet1.txt", W)

In [2]:
def checkCalculations():

    img_width = 224
    img_height = 224

    img = image.load_img(fileName, target_size=(img_width, img_height))
    X = image.img_to_array(img)

    # Normalized the Input
    X = tf.keras.utils.normalize(X, axis=2)
    
    # Set as 1
    #X[:,:,0] = 1
    #X[:,:,1] = 1
    #X[:,:,2] = 1

    # Set as Combination
    #X[:,:,0] = 1
    #X[:,:,1] = 10
    #X[:,:,2] = 100

    print(X[0][0][0])
    print(X[0][1][0])
    print(X[0][2][0])
    X = np.expand_dims(X, axis=0)
    print(X.shape)

    # Weights part
    W = model.layers[2].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[2].set_weights(W)
    
    
    W = model.layers[5].get_weights()[0]
    # Weights set as 1
    #W[:,:,:,:] = 1
    W = np.expand_dims(W, axis=0)
    model.layers[5].set_weights(W)
    
    
    output = model.predict(X)
    print("Output Shape ---> ")
    print(output.shape)
    print("--------------------------------")

    # Save Output to file
    fOutput = open("data/SecondLayer/Second_Layer_Output.txt", "w")
    for i in range(len(output)):
        for j in range(len(output[0][0][0])):
            for k in range(len(output[0])):
                for l in range(len(output[0][0])):
                    fOutput.write(str(output[i][k][l][j]) + "\n")
    fOutput.close()
    print("Output File Writing Complete!!!")

    print("\n\n Description of Layers")
    for layer in model.layers:
        print(layer)


In [3]:
import time
start = time.time()
checkCalculations()
end = time.time()
print("Total Time -->", (end - start))


/home/nikhil/anaconda3/lib/python3.6/site-packages/numpy/linalg/linalg.py:2198: RuntimeWarning: invalid value encountered in sqrt
  return sqrt(add.reduce(s, axis=axis, keepdims=keepdims))


0.145017
0.351735
0.563547
(1, 224, 224, 3)
Output Shape ---> 
(1, 112, 112, 32)
--------------------------------
Output File Writing Complete!!!


 Description of Layers
Total Time --> 2.1179473400115967


In [ ]:
def checkFile_FirstLayer():
    fExp = open('data/FirstLayer/First_Layer_Output.txt', 'r')
    fAct = open('data/FirstLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [ ]:
checkFile_FirstLayer()

In [10]:
def checkFile_SecondLayer():
    fExp = open('data/SecondLayer/Second_Layer_Output.txt', 'r')
    fAct = open('data/SecondLayer/output.txt', 'r')
    
    cExp = fExp.readlines()
    cAct = fAct.readlines()
    counter = 0
    for i in range(len(cExp)):
         if int(round(float(cAct[i].strip()))) != int(round(float(cExp[i].strip()))):
            counter += 1
            print(str(cAct[i].strip()) + "--->" + str(cExp[i].strip()) )
        
    print("Number of mismatch - " + str(counter))
    
    fExp.close()
    fAct.close()
    
        

In [11]:
checkFile_SecondLayer()

31.500059--->31.4999
8.500112--->8.49998
10.500037--->10.4999
10.500033--->10.4999
-16.500052--->-16.5
2.500264--->2.49992
2.500061--->2.49971
83.499977--->83.5
1.499952--->1.5
-1.499985--->-1.50017
10.500051--->10.5
10.500028--->10.5
-20.500074--->-20.5
-16.500048--->-16.5
-4.500028--->-4.49997
1.500022--->1.4998
1.500050--->1.49986
1.500148--->1.49994
1.500104--->1.49977
1.500014--->1.49984
1.500090--->1.49993
1.500041--->1.49992
1.500112--->1.49992
1.500006--->1.49979
1.500134--->1.49996
1.500139--->1.49993
1.500017--->1.49984
1.500014--->1.49983
1.500047--->1.49987
1.500151--->1.49997
Number of mismatch - 30
